In [1]:
import os
import sys
import inspect
import tqdm
import warnings
import argparse
import datetime

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir) + "/src/"
sys.path.insert(0, parentdir) 
from modeling import *
from utils import NpEncoder
%load_ext autoreload
%autoreload 2


In [2]:
type_ = "R"

# load data and get player
datafile = "../data/raw/data_%s.json"%(type_)
print("data file\t", datafile)
with open(datafile, 'r') as f:
    data = json.load(f)
player = list(data.keys())[0]


# multi-start parameters
params = {
"inv_temp": 0.1, 
}

data file	 ../data/raw/data_R.json


In [3]:
filter_fn = filter_depth
value_fn = value_max
games = format_games(data[player]["data"])

In [4]:
model = Model(filter_fn, value_fn, variant =type_)

In [5]:
game_data = preprocess_data(games)
pov_array = make_pov_array(game_data.boards, game_data.paths)
filter_pov_array = model.filter_fn(pov_array)
ml, ml_params = model.max_likelihood_estimation(params, filter_pov_array, game_data.choose_left)

In [51]:
choose_left = game_data.choose_left
p_left = model.get_p_left(params, filter_pov_array)
log_likelihood = choose_left * np.log(p_left) + (1 - choose_left) * np.log(1 - p_left)
log_likelihood_player = log_likelihood.sum(["games", "trials", "conditions"], keep_attrs = ["conditions"])
ml = log_likelihood_player.max("filter_params")
# mle = argmax_random_tiebreaker(log_likelihood_player)
# copy_and_update(params, {"filter_params": mle})

In [11]:
values = pov_array.sel(games = 0, conditions = 0)
left = values.isel(rows = values.rows[1:], cols = values.cols[:-1])
right = values.isel(rows = values.rows[1:], cols = values.cols[1:])

In [25]:
left.where(left.rows >= left.cols).mean("cols")

<xarray.DataArray (trials: 7, rows: 7)> Size: 392B
array([[9.        , 6.5       , 4.66666667, 6.        , 3.6       ,
        6.5       , 4.57142857],
       [7.        , 6.        , 6.33333333, 4.        , 7.        ,
        5.16666667, 0.        ],
       [7.        , 6.5       , 3.33333333, 6.75      , 5.2       ,
        0.        , 0.        ],
       [6.        , 4.        , 6.33333333, 5.        , 0.        ,
        0.        , 0.        ],
       [5.        , 5.        , 6.        , 0.        , 0.        ,
        0.        , 0.        ],
       [6.        , 7.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [7.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])
Coordinates:
    conditions  float64 8B 0.0
Dimensions without coordinates: trials, rows

In [26]:
np.array([6.,  5.,  3.,  2.,  2]).mean()

np.float64(3.6)

In [21]:
left.where(left.rows >= left.cols)

<xarray.DataArray (trials: 7, rows: 7, cols: 7)> Size: 3kB
array([[[ 9., nan, nan, nan, nan, nan, nan],
        [ 7.,  6., nan, nan, nan, nan, nan],
        [ 5.,  7.,  2., nan, nan, nan, nan],
        [ 6.,  6.,  7.,  5., nan, nan, nan],
        [ 6.,  5.,  3.,  2.,  2., nan, nan],
        [ 8.,  4.,  6.,  9.,  8.,  4., nan],
        [ 5.,  4.,  7.,  7.,  2.,  6.,  1.]],

       [[ 7., nan, nan, nan, nan, nan, nan],
        [ 5.,  7., nan, nan, nan, nan, nan],
        [ 6.,  6.,  7., nan, nan, nan, nan],
        [ 6.,  5.,  3.,  2., nan, nan, nan],
        [ 8.,  4.,  6.,  9.,  8., nan, nan],
        [ 5.,  4.,  7.,  7.,  2.,  6., nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[ 7., nan, nan, nan, nan, nan, nan],
        [ 6.,  7., nan, nan, nan, nan, nan],
        [ 5.,  3.,  2., nan, nan, nan, nan],
        [ 4.,  6.,  9.,  8., nan, nan, nan],
...
        [ 0.,  0.,  0.,  0., nan, nan, nan],
        [ 0.,  0.,  0.,  0.,  0., nan, nan],
        [ 0.,  0.,  0.,  0.,  0.,  0., nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[ 6., nan, nan, nan, nan, nan, nan],
        [ 7.,  7., nan, nan, nan, nan, nan],
        [ 0.,  0.,  0., nan, nan, nan, nan],
        [ 0.,  0.,  0.,  0., nan, nan, nan],
        [ 0.,  0.,  0.,  0.,  0., nan, nan],
        [ 0.,  0.,  0.,  0.,  0.,  0., nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[ 7., nan, nan, nan, nan, nan, nan],
        [ 0.,  0., nan, nan, nan, nan, nan],
        [ 0.,  0.,  0., nan, nan, nan, nan],
        [ 0.,  0.,  0.,  0., nan, nan, nan],
        [ 0.,  0.,  0.,  0.,  0., nan, nan],
        [ 0.,  0.,  0.,  0.,  0.,  0., nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]]])
Coordinates:
    conditions  float64 8B 0.0
Dimensions without coordinates: trials, rows, cols

In [22]:
(5 + 7 + 2) / 3

4.666666666666667